*Note: This notebook requires a node with sufficient processing power. Google Colab randomly assigns a GPU to its users so this notebook may fail (produce a white image or nothing at all), depending on the Demi-Gods and Semi-Devils at Google. Just try again later. We will try and make it work on all nodes in the future. Subscribing to Colab Pro solves this problem.*


*Berghain Queue*

<img src="https://pollinations.ai/ipfs/QmbkKienueKJvnfxYCMNpaa3kcHeu3DB9e3NZkwGLYQady/image.png?filename=frame_0500.png" width="300"/>



In [ ]:
prompts = "Beirut Skyline. #pixelart" #@param {type:"string"}
output_path = "/content/output"

In [ ]:
#@title Setup

#@markdown Please execute this cell by pressing the _Play_ button 
#@markdown on the left. For setup,
#@markdown **you need to run this cell,
#@markdown then choose Runtime -> Restart Runtime from the menu,
#@markdown and then run the cell again**. It should remind you to
#@markdown do this after the first run.

#@markdown Setup can take 5-10 minutes, but once it is complete it usually does not need to be repeated
#@markdown until you close the window.

#@markdown **Note**: This installs the software on the Colab 
#@markdown notebook in the cloud and not on your computer.

# https://stackoverflow.com/a/56727659/1010653

from IPython.utils import io
with io.capture_output() as captured:
  !git clone https://github.com/openai/CLIP
  # !pip install taming-transformers
  !git clone https://github.com/CompVis/taming-transformers.git
  !rm -Rf clipit
  !git clone https://github.com/pollinations/clipit
  !pip install ftfy regex tqdm omegaconf pytorch-lightning
  !pip install kornia
  !pip install imageio-ffmpeg   
  !pip install einops
  !pip install torch-optimizer
  !pip install easydict
  !pip install braceexpand
  !pip install git+https://github.com/pvigier/perlin-numpy

  # ClipDraw deps
  !pip install svgwrite
  !pip install svgpathtools
  !pip install cssutils
  !pip install numba
  !pip install torch-tools
  !pip install visdom

  !git clone https://github.com/BachiLi/diffvg
  %cd diffvg
  # !ls
  !git submodule update --init --recursive
  !python setup.py install
  %cd ..

import sys
sys.path.append("clipit")

result_msg = "setup complete"
import IPython
import os
if not os.path.isfile("first_init_complete"):
  # put stuff in here that should only happen once
  !mkdir -p models
  os.mknod("first_init_complete")
  result_msg = "Please choose Runtime -> Restart Runtime from the menu, and then run Setup again"

js_code = f'''
document.querySelector("#output-area").appendChild(document.createTextNode("{result_msg}"));
'''
js_code += '''
for (rule of document.styleSheets[0].cssRules){
  if (rule.selectorText=='body') break
}
rule.style.fontSize = '30px'
'''
display(IPython.display.Javascript(js_code))

In [ ]:
#@title Settings

#@markdown Enter a description of what you want to draw - I usually add #pixelart to the prompt.
#@markdown If PixelDraw is not used, it will use VQGAN instead.
#@markdown <br>

aspect = "widescreen" #@param ["widescreen", "square"]

use_pixeldraw = True #@param {type:"boolean"}

#@markdown When you have the settings you want, press the play button on the left.
#@markdown The system will save these and start generating images below.

#@markdown When that is done you can change these
#@markdown settings and see if you get different results. Or if you get
#@markdown impatient, just select "Runtime -> Interrupt Execution".
#@markdown Note that the first time you run it may take a bit longer
#@markdown as nessary files are downloaded.


#@markdown
#@markdown *Advanced: you can also edit this cell and add add additional
#@markdown settings, combining settings from different notebooks.*


# Simple setup
import clipit

# these are good settings for pixeldraw
clipit.reset_settings()
clipit.add_settings(prompts=prompts, aspect=aspect)
clipit.add_settings(quality="better", scale=2.5)
clipit.add_settings(use_pixeldraw=use_pixeldraw)

#### YOU CAN ADD YOUR OWN CUSTOM SETTING HERE ####
# this is the example of how to run longer with less frequent display
clipit.add_settings(iterations=350, display_every=50, save_every=20, make_video=True, output=output_path+"/output.png")

settings = clipit.apply_settings()
clipit.do_init(settings)
clipit.do_run(settings)
!sleep 10

In [ ]:
out_file=output_path+"/video.mp4"

!mkdir -p /tmp/ffmpeg
!cp $output_path/*.png /tmp/ffmpeg
last_frame=!ls -t /tmp/ffmpeg/*.png | head -1
last_frame = last_frame[0]

# Copy last frame to start and duplicate at end so it sticks around longer
end_still_seconds = 4
!cp -v $last_frame /tmp/ffmpeg/0000.png
for i in range(end_still_seconds * 10):
  pad_file = f"/tmp/ffmpeg/zzzz_pad_{i:05}.png"
  !cp -v $last_frame $pad_file

!ffmpeg  -r 10 -i /tmp/ffmpeg/%*.png -y -c:v libx264 -vf "scale=800:512" /tmp/vid_no_audio.mp4
!ffmpeg -i /tmp/vid_no_audio.mp4 -f lavfi -i anullsrc -c:v copy -c:a aac -shortest -y "$out_file"

print("Written", out_file)
!sleep 2
!rm -r /tmp/ffmpeg